In [1]:
import sys
from pathlib import Path

import h5py as h5
import numpy as np
import pandas as pd

sys.path.append('../lib')
from storage import get_storage_functions
from local_paths import preproc_dir

# Parameters

In [2]:
#============================================================================
# session
#============================================================================
sess_name = 'sess_name'


#============================================================================
# main
#============================================================================
# fixation selection
min_fix_dur  = 100   # ms
within_image = True  # whther fixation must land within image bounds
t_margin     = 500   # ms; make sure neural data covers fixation periods

# saccade selection
min_sacc_size =    4  # dva; only consider saccades at least this large
# - make sure any other fixation(s) closer than `min_sep_win` from the saccade
#   are also at least `min_sacc_size` apart across the saccade
min_sep_win   =  100  # ms


#============================================================================
# paths
#============================================================================
# this script will write to preproc file
proc_dir = preproc_dir
overwrite = False  # overwrite any existing results; use with caution

In [3]:
# Parameters
sess_name = "Pa210201"


# Check parameters and whether already done

In [4]:
proc_path = Path(proc_dir) / (sess_name + '-proc.h5')
print('Loading shared processing from', proc_path)
proc_path = proc_path.expanduser()
assert proc_path.is_file()

output_path = proc_path

Loading shared processing from ../test_data/Pa210201-proc.h5


In [5]:
if output_path.is_file():
    if overwrite:
        with h5.File(output_path, 'a') as f:
            for g in ('fixation_selection', 'saccade_selection', 'progress_report/select_fix_sacc/all_done'):
                if g in f:
                    del f[g]
    else:
        with h5.File(output_path, 'r') as f:
            try:
                if f['progress_report/select_fix_sacc/all_done'][()].item():
                    raise RuntimeError(f'{sess_name} has already been processed')
            except KeyError:
                pass

In [6]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path, overwrite=overwrite)

In [7]:
fix_df = pd.read_hdf(proc_path, 'fixation_dataframe', 'r')
with h5.File(proc_path, 'r') as f:
    duration = f['recording_duration'][()]
    im_size_dva = f['stimulus/size_dva'][()]

# Select fixations

In [8]:
fix_sel_mask = (
    (fix_df['Duration'] >= min_fix_dur)
    & (fix_df['Time'] >= t_margin)
    & (fix_df['End time'] <= (duration-t_margin))
).values
print(f'from {len(fix_df)} fixations, selecting {fix_sel_mask.sum()} '
      f'with duration >= {min_fix_dur} ms and within recording bounds')

# fixation must land within image bounds
if within_image:
    xys = fix_df[['Relative X', 'Relative Y']].values.astype(float)
    d2b = np.array([
        np.max([xys[:,j]-im_size_dva[j]/2, -im_size_dva[j]/2-xys[:,j]], axis=0)
        for j in range(2)
    ])  # shape (2, n_fix)
    m = fix_sel_mask & np.all(d2b<0, axis=0)  # shape (n_fix,)
    print(f'from {fix_sel_mask.sum()} fixations, selecting {m.sum()} '
          f'that are fully within image bounds (size: {im_size_dva})')
    fix_sel_mask = m

fix_sel = np.nonzero(fix_sel_mask)[0]
print(f'selected {len(fix_sel)} fixations')

from 6927 fixations, selecting 6215 with duration >= 100 ms and within recording bounds
from 6215 fixations, selecting 5979 that are fully within image bounds (size: [16. 16.])
selected 5979 fixations


In [9]:
save_results('fixation_selection/min_fix_dur', min_fix_dur, attrs=dict(unit='ms'))
save_results('fixation_selection/within_image', within_image)
save_results('fixation_selection/t_margin', t_margin, attrs=dict(unit='ms'))
save_results('fixation_selection/fixation_indices', fix_sel)  # shape (nfix,) indexing into fix_df

# Select saccades

In [10]:
# start with selected fixations
fix2_sel = fix_sel.copy()

# 1. have a previous fixation
m = fix_df.iloc[fix2_sel]['Preceding fixation index'].values >= 0
fix2_sel = fix2_sel[m]
fix1_sel = fix_df.iloc[fix2_sel]['Preceding fixation index'].values.astype(fix2_sel.dtype)
print(f'from {m.size} fixations, selecting {fix2_sel.size} with a previous fixation')

# 2. previous fixation must also be selected
m = pd.Series(fix1_sel).isin(fix_sel).values
fix2_sel = fix2_sel[m]
fix1_sel = fix1_sel[m]
print(f'from {len(m)} saccades, selecting {len(fix2_sel)} '
      f'with starting fixations that also match selection criteria')

sacc_vecs_sel = (
    fix_df.iloc[fix2_sel][['Relative X', 'Relative Y']].values
    - fix_df.iloc[fix1_sel][['Relative X', 'Relative Y']].values)
sacc_sizes = np.linalg.norm(sacc_vecs_sel, axis=-1)

sacc_sel = np.array([fix1_sel, fix2_sel])
print(f'selected {sacc_sel.shape[1]} saccades')

from 5979 fixations, selecting 4607 with a previous fixation
from 4607 saccades, selecting 4037 with starting fixations that also match selection criteria
selected 4037 saccades


### Further select large saccades

In [11]:
# 3. saccade size must be large
m = sacc_sizes >= min_sacc_size
sel = np.nonzero(m)[0]
print(f'from {len(m)} saccades, selecting {len(sel)} >= {min_sacc_size} dva')

# 4. nearby fixations must also be at least `min_sacc_size` apart across the saccade
#    this check is redundant if min_sep_win <= min_fix_dur, but check just in case
min_seps = sacc_sizes[m].copy()
for i, ((itr, ifix), row) in enumerate(fix_df.iloc[fix2_sel[sel]].iterrows()):
    t0 = fix_df.iloc[fix1_sel[sel[i]]]['End time']  # saccade start time
    t1 = row['Time']  # saccade stop time
    tr_fixs = fix_df.loc[(itr, slice(None))]

    m0 = tr_fixs.index < ifix - 1
    if not m0.any(): continue
    m0[m0] = tr_fixs.loc[m0, 'End time'].values > (t0 - min_sep_win)
    if not m0.any(): continue

    m1 = tr_fixs.index > ifix
    if not m1.any(): continue
    m1[m1] = tr_fixs.loc[m1, 'Time'] <= (t1 + min_sep_win)
    if not m1.any(): continue

    pwd = np.linalg.norm(
        tr_fixs.loc[m0, ['Relative X', 'Relative Y']].values[:,None,:]
        - tr_fixs.loc[m1, ['Relative X', 'Relative Y']].values[None,:,:],
        axis=-1)
    min_seps[i] = pwd.min()

submask = m.copy()
submask[submask] = m = min_seps >= min_sacc_size
print(f'from {len(sel)} saccades, selecting {submask.sum()} whose pre-/post-fixations '
      f'(within {min_sep_win} ms of saccade) are also at least {min_sacc_size} dva apart')

from 4037 saccades, selecting 2399 >= 4 dva


from 2399 saccades, selecting 2399 whose pre-/post-fixations (within 100 ms of saccade) are also at least 4 dva apart


In [12]:
link_dsets('saccade_selection/fixation_selection', 'fixation_selection')
save_results('saccade_selection/fixation_indices', sacc_sel)  # shape (2, nsacc) indexing into fix_df
save_results('saccade_selection/saccade_sizes', sacc_sizes, attrs=dict(unit='dva'))  # shape (nsacc,)

save_results('saccade_selection/large/min_sacc_size', min_sacc_size, attrs=dict(unit='dva'))
save_results('saccade_selection/large/min_sep_win', min_sep_win, attrs=dict(unit='ms'))
save_results('saccade_selection/large/saccade_subset', submask)  # shape (nsacc,) binary selection into sacc_sel
save_results('saccade_selection/large/saccade_sizes', sacc_sizes[submask], attrs=dict(unit='dva'))  # shape (nsacc_lg,)
save_results('saccade_selection/large/min_seperations', min_seps[m], attrs=dict(unit='dva'))  # shape (nsacc_lg,)

# Wrap up

In [13]:
save_results('progress_report/select_fix_sacc/all_done', True)

In [14]:
%load_ext watermark
%watermark -vm --iversions -rbg

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.12.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-92-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

Git hash: b0bb10f45dee065cc3af96fe224326b883d27431

Git repo: https://github.com/willwx/free_viewing_staging.git

Git branch: master

h5py  : 3.8.0
numpy : 1.24.3
sys   : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
pandas: 2.0.1

